In [ ]:
%%capture
!pip install openmined_psi

In [ ]:
import syft as sy
duet = sy.join_duet(loopback=True)

In [ ]:
import openmined_psi as psi

In [ ]:
duet.store.pandas

In [ ]:
# from: https://github.com/OpenMined/PSI/blob/master/private_set_intersection/python/tests.py

In [ ]:
reveal_intersection_ptr = duet.store["reveal_intersection"]

In [ ]:
reveal_intersection = reveal_intersection_ptr.get(
    request_block=True,
    name="reveal_intersection",
    reason="Are we revealing or not?",
    timeout_secs=10,
    delete_obj=False
)
reveal_intersection

In [ ]:
client = psi.client.CreateWithNewKey(reveal_intersection)

In [ ]:
client_items = ["Element " + str(i) for i in range(1000)]
sy_client_items_len = sy.lib.python.Int(len(client_items)).tag("client_items_len")
sy_client_items_len_ptr = sy_client_items_len.send(duet, searchable=True)

In [ ]:
setup_ptr = duet.store["setup"]

In [ ]:
server_ptr = duet.store["server"]

# TODO make setup serializable
setup = setup_ptr.get(
    request_block=True,
    name="setup",
    reason="To get the setup",
    timeout_secs=10,
)

In [ ]:
request = client.CreateRequest(client_items)
# TODO: make openmined_psi.proto_request serializable
response_ptr = server_ptr.ProcessRequest(request)

In [ ]:
# TODO make openmined_psi.proto_response serializable
response = response_ptr.get(
    request_block=True,
    name="reponse",
    reason="To get the result",
    timeout_secs=10,
)

In [ ]:
if reveal_intersection:
    intersection = client.GetIntersection(setup, response)
    iset = set(intersection)
    for idx in range(len(client_items)):
        if idx % 2 == 0:
            assert idx in iset
        else:
            assert idx not in iset

In [ ]:
if not reveal_intersection:
    intersection = client.GetIntersectionSize(setup, response)
    assert intersection >= (len(client_items) / 2.0)
    assert intersection <= (1.1 * len(client_items) / 2.0)

In [ ]:
duet.store.pandas